# 第 5 章 
## 第 2 节 市场环境
在上一节中，我们看了市场中最重要的State类，分钟线市场环境类MinuteBarEnv只是在其基础上进行了封装。
### 5.2.1. 市场环境构造函数
我们先来看其构造函数：

In [ ]:
class MinuteBarEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    spec = EnvSpec("StocksEnv-v0")

    def __init__(self, prices, bars_count=AppConfig.DEFAULT_BARS_COUNT,
                 commission=AppConfig.DEFAULT_COMMISSION_PERC,
                 reset_on_close=True, state_1d=False,
                 random_ofs_on_reset=True, reward_on_close=False,
                 volumes=False):
        assert isinstance(prices, dict)
        self._prices = prices
        if state_1d:
            self._state = State1D(
                bars_count, commission, reset_on_close,
                reward_on_close=reward_on_close, volumes=volumes)
        else:
            self._state = State(
                bars_count, commission, reset_on_close,
                reward_on_close=reward_on_close, volumes=volumes)
        self.action_space = gym.spaces.Discrete(n=len(AssetActions))
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf,
            shape=self._state.shape, dtype=np.float32)
        self.random_ofs_on_reset = random_ofs_on_reset
        self.seed()
        
    def test_MinuteBarEnv_main(self):
        '''
        研究市场环境类
        '''
        year = 2016
        instrument = 'data\\YNDX_160101_161231.csv'
        stock_data = BarData.load_year_data(year)
        print('stock_data: {0};'.format(stock_data[instrument]))
        env = MinuteBarEnv(
                stock_data, bars_count=AppConfig.BARS_COUNT, volumes=True)
        print('action_sapce = {0};'.format(env.action_space))
        print('shape: {0};'.format(env._state.shape))

代码解读如下所示：
* 第11行：self._prices的格式为：
```
stock_data: BarPrices(open=array([1156.9, 1150.6, 1150.2, ..., 1245.5, 1246. , 1244. ], dtype=float32), high=array([0.00086438, 0.        , 0.        , ..., 0.        , 0.        ,
       0.00361736], dtype=float32), low=array([-0.0033711 , -0.00017378, -0.00060855, ..., -0.00080289,
       -0.00160514, -0.00040193], dtype=float32), close=array([-0.0033711 , -0.00017378, -0.00043471, ..., -0.00080289,
       -0.00080257,  0.00361736], dtype=float32), volume=array([ 43.,   5., 165., ..., 200., 231., 191.], dtype=float32));
```
* 第12$\sim$19行：生成第1节中的状态对象；
* 行动空间为0、1、2三个，因为AssetActions为枚举类型：Keep、Buy、Sell；
* 观察状态为42维数组；
* 每次重置系统，开始点为随机的，这样可以增加样本数，增高网络泛化能力；

### 5.2.2. 市场环境重置
市场环境类MinutBarEnv类的reset函数为：

In [ ]:
class MinuteBarEnv(gym.Env):
    def reset(self):
        # make selection of the instrument and it's offset. Then reset the state
        self._instrument = self.np_random.choice(
            list(self._prices.keys()))
        prices = self._prices[self._instrument]
        bars = self._state.bars_count
        if self.random_ofs_on_reset:
            offset = self.np_random.choice(
                prices.high.shape[0]-bars*10) + bars
        else:
            offset = bars
        self._state.reset(prices, offset)
        return self._state.encode()
    
    def test_MinuteBarEnv_main(self):
        '''
        研究市场环境类
        '''
        year = 2016
        instrument = 'data\\YNDX_160101_161231.csv'
        stock_data = BarData.load_year_data(year)
        print('stock_data: {0};'.format(stock_data[instrument]))
        env = MinuteBarEnv(
                stock_data, bars_count=AppConfig.BARS_COUNT, volumes=True)
        obs = env.reset()
        print('observation: {0};'.format(obs))

代码解读如下所示：
* 第4行：随机从相对最高价、相对最低价、相对收盘价、交易量序列中选择一个作为数据集；
* 第8$\sim$12行：随机选择开始交易的日期，这样同一个序列可以视为很多个不同的数据集；
* 第13行：重置市场环境的State类实例；
* 第14行：调用State.encode方法产生代理可观测的状态obs；

### 5.2.3. 市场环境运行
在市场环境初始化之后，我们就可以让市场环境运行起来，代码如下所示：

In [ ]:
class MinuteBarEnv(gym.Env):
    def step(self, action_idx):
        action = AssetActions(action_idx)
        reward, done = self._state.step(action)
        obs = self._state.encode()
        info = {
            "instrument": self._instrument,
            "offset": self._state._offset
        }
        return obs, reward, done, info

    def render(self, mode='human', obs=None, reward=0.0, info={}, close=False):
        print('打印信息: {0};'.format(obs))

    def test_MinuteBarEnv_main(self):
        '''
        研究市场环境类
        '''
        year = 2016
        instrument = 'data\\YNDX_160101_161231.csv'
        stock_data = BarData.load_year_data(year)
        print('stock_data: {0};'.format(stock_data[instrument]))
        env = MinuteBarEnv(
                stock_data, bars_count=AppConfig.BARS_COUNT, volumes=True)
        obs = env.reset()
        seq = 1
        while True:
            if seq > 3:
                done = True
            if 1 == seq:
                action = AssetActions.Buy
            elif 2 == seq:
                action = AssetActions.Sell
            else:
                action = AssetActions.Keep
            obs, reward, done, info = env.step(action)
            if done:
                break
            env.render(mode='human', obs=obs, reward=reward, info=info)
            seq += 1
        print('observation: {0};'.format(obs))

在以上代码中，我们需要重写render方法，可以通过可视化方法来展现具体的交易过程。